In [102]:
from __future__ import print_function
import requests
import pandas as pd
import numpy as np

In [103]:
kewy_list="T:\colin\BRice\V2\gwas-gene-discovery\mock_keyword_list.txt"
disc_genes="T:\colin\BRice\V2\gwas-gene-discovery\MLM.DTF.GWAS.Results\summary_genes_discovered.txt"

In [108]:
def knetapi(disc_genes):
    with open(kewy_list, "r") as fl:
        pheno=[]
        for line in fl:
            pheno.append(line.rstrip())
            #all keyword from plain text file into pheno list.
        summary=pd.read_csv(disc_genes, sep="\t")
        genelist=list(summary[u'GENE'])
        genestr=(",").join(genelist)
        #all genes in column 0 of summary into one string
        
        #use str.join() to convert multiple elments in a list into one string.
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        link="http://knetminer.rothamsted.ac.uk/riceknet/genome?"
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)
        
        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()

        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
        
        #delete useless columns from genetable.
        genetable=genetable.drop([u'ONDEX-ID', u'GENE NAME', u'TAXID', u'USER', u'QTL'], axis=1)
        
        #rename genetable columns.
        genetable.rename (
        columns={
            "ACCESSION":"Knetminer_genes",
            "SCORE":"KnetScore"
        }, inplace=True
        )
        
        knetgenes=list(genetable[u'Knetminer_genes'])
        knetscores=list(genetable[u'KnetScore'])
        
        #print(knetgenes)
        #print(knetscores)
        
        knetdict=dict(zip(knetgenes, knetscores))
        #print(knetdict)
        #print(knetdict[u'OS03G0122600'])
        #compare genetable and summary
        ordered_score=[]
        for i in summary[u'GENE']:
            i=i.upper()
            print(i)
            print(knetdict[u'{}'.format(i)])
            ordered_score.append(knetdict[u'{}'.format(i)])
        print(ordered_score)
        summary[u'SCORE'] = ordered_score
        return(summary)
        summary.to_csv(disc_genes)

In [109]:
knetapi(disc_genes)

OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS03G0122600
125.60
OS06G0164900
5.94
OS03G0122600
125.60
OS06G0166100
0.00
OS06G0156700
10.64
OS06G0156600
10.64
[u'125.60', u'125.60', u'125.60', u'125.60', u'125.60', u'5.94', u'125.60', u'0.00', u'10.64', u'10.64']


,GENE,CHROMOSOME,SNP_number,SNP_Base_Position,p-value,logP,SNP_effect,allele_info,Gene_description,strand,SCORE
0,Os03g0122600,3,1,1278045,8.740000e-09,8.058489,intron_variant,"['G', 'A']",None,None,125.60
1,Os03g0122600,3,2,1283193,2.140000e-08,7.669586,intron_variant,"['C', 'T']",None,None,125.60
2,Os03g0122600,3,3,1275151,2.980000e-08,7.525784,intron_variant,"['T', 'C']",None,None,125.60
3,Os03g0122600,3,4,1286390,4.790000e-08,7.319664,intron_variant,"['G', 'A']",None,None,125.60
4,Os03g0122600,3,5,1300254,5.040000e-08,7.297569,5_prime_UTR_variant,"['C', 'T']",None,None,125.60
5,Os06g0164900,6,6,3287871,5.290000e-08,7.276544,missense_variant,"['A', 'T']",None,None,5.94
6,Os03g0122600,3,7,1272815,7.360000e-08,7.133122,intron_variant,"['G', 'A']",None,None,125.60
7,Os06g0166100,6,8,3326064,1.020000e-07,6.991400,synonymous_variant,"['T', 'A']",None,None,0.00
8,Os06g0156700,6,11,2898059,3.070000e-07,6.512862,upstream_gene_variant,"['A', 'G']",None,None,10.64
9,Os06g0156600,6,13,2894959,6.850000e-07,6.164309,intron_variant,"['T', 'C']",None,None,10.64
